##  Departamento de Cómputo en la Nube
---



In [22]:
#!pip install azureml-core

In [23]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
id = open('my_id.json', 'r')
mi = json.load(id)
my_id = mi["my_id"],

from azureml.core import Workspace

# ws = Workspace.create(name="workspace",
#                       subscription_id = my_id,
#                       resource_group = "resource_group2",
#                       location = "centralindia")

ws = Workspace.get(name="my_first_aml_workspace",
                      subscription_id = mi["my_id"],
                      resource_group = "Azure_for_Students",
                      location = "eastus") 

In [24]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [25]:
################ score.py ###################

scorepy = """import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('model')
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['datos']
        data = pd.DataFrame(data)

        # Seleccionar columnas relevantes
        data = data[['CompanyName', 'SalesPerson', 'ModifiedDate']]

        # Convertir ModifiedDate a datetime y luego a rango de días
        data['ModifiedDate'] = pd.to_datetime(data['ModifiedDate'])
        data['Fecha'] = (data['ModifiedDate'] - data['ModifiedDate'].min()).dt.days

        # Eliminar columna original
        data.drop(columns=['ModifiedDate'], inplace=True)

        # Realizar predicción (ajusta aquí si es necesario)
        result = model.predict(data[['CompanyName', 'SalesPerson']])

        return json.dumps({"result": result.tolist()})
    except Exception as e:
        return json.dumps({"error": str(e)})
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [26]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("regression-env-v1")
deps = CondaDependencies()
deps.set_python_version("3.9")

deps.add_pip_package("azureml-defaults")
deps.add_pip_package("pandas==2.2.2")
deps.add_pip_package("numpy==1.26.4")
deps.add_pip_package("joblib==1.2.0")
deps.add_pip_package("scikit-learn==1.2.0")  
env.python.conda_dependencies = deps
env.register(workspace=ws)


{
    "assetId": "azureml://locations/eastus/workspaces/c49fa2b4-c7cc-4964-ba47-364e091933da/environments/regression-env-v1/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "regression-env-v1",
    "python"

In [27]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(environment=env,entry_script="score.py",)
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(
    workspace=ws,
    name='regression-service-v1',
    models=[registered_model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True,
)

/var/folders/7b/g3f6kzfd1b99b918vd8tvd100000gn/T/ipykernel_48247/1706559560.py:7: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


In [28]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-23 22:24:00-06:00 Creating Container Registry if not exists.
2025-04-23 22:24:01-06:00 Use the existing image.
2025-04-23 22:24:01-06:00 Generating deployment configuration.
2025-04-23 22:24:03-06:00 Submitting deployment to compute.
2025-04-23 22:24:06-06:00 Checking the status of deployment regression-service-v1..
2025-04-23 22:28:48-06:00 Checking the status of inference endpoint regression-service-v1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [29]:
scoring_uri = service.scoring_uri
#print(f"URI del servicio: {scoring_uri}")

In [31]:
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()